In [13]:
from gensim.models import Word2Vec
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from itertools import combinations
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
data = data.iloc[0:100]

In [0]:
def uni(stri):
  word_filtered = []
  stri = stri.translate(str.maketrans('', '', string.punctuation))
  words = stri.split()
  for w in words:
    if w not in stop_words:
        word_filtered.append(w)
  unique = list(dict.fromkeys(word_filtered))
  return unique

In [5]:
data['unique'] = data['reviewText'].apply(uni)
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,unique
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004","[I, bought, first, HP12C, 1984, served, faithf..."
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007","[WHY, THIS, BELATED, REVIEW, I, feel, obliged,..."
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011","[I, HP, 48GX, kicking, twenty, years, 11, 25, ..."
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006","[Ive, started, finance, stuff, recently, went,..."
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013","[For, simple, calculations, discounted, cash, ..."


In [6]:
bag = []
for i in range(data.shape[0]):
  bag += data['unique'][i]
len(bag)

5707

In [7]:
bag_unique = list(dict.fromkeys(bag))
bag_unique = [x.lower() for x in bag_unique]
len(bag_unique)

2466

In [8]:
comb = list(combinations(bag_unique, 2))
len(comb)

3039345

In [18]:
from nltk.util import ngrams
corp = list(ngrams(bag_unique, 5))
corp[0]

('i', 'bought', 'first', 'hp12c', '1984')

In [0]:
model = Word2Vec(corp, size=100, window=5, min_count=1, workers=4)

In [20]:
similarity = []
for i in range(len(comb)):
  similarity.append(model.similarity(comb[i][0], comb[i][1]))
len(similarity)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


3039345

In [21]:
final = pd.DataFrame(list(zip(comb, similarity)), columns=['Combinations', 'Similarity_score'])
final.head()

,Combinations,Similarity_score
0,"(i, bought)",0.165660
1,"(i, first)",0.052574
2,"(i, hp12c)",0.193158
3,"(i, 1984)",-0.052973
4,"(i, served)",-0.000501
